# Test smple_weight, kernels, C, sklearn estimator

# Setup
Uncomment the next cell if STree is not already installed

In [1]:
#
# Google Colab setup
#
#!pip install git+https://github.com/doctorado-ml/stree

In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification, load_iris, load_wine
from sklearn.model_selection import train_test_split
from stree import Stree
import time

In [3]:
import os
if not os.path.isfile('data/creditcard.csv'):
    !wget --no-check-certificate --content-disposition http://nube.jccm.es/index.php/s/Zs7SYtZQJ3RQ2H2/download
    !tar xzf creditcard.tgz

In [4]:
random_state=1

def load_creditcard(n_examples=0):
    import pandas as pd
    import numpy as np
    import random
    df = pd.read_csv('data/creditcard.csv')
    print("Fraud: {0:.3f}% {1}".format(df.Class[df.Class == 1].count()*100/df.shape[0], df.Class[df.Class == 1].count()))
    print("Valid: {0:.3f}% {1}".format(df.Class[df.Class == 0].count()*100/df.shape[0], df.Class[df.Class == 0].count()))
    y = df.Class
    X = df.drop(['Class', 'Time', 'Amount'], axis=1).values
    if n_examples > 0:
        # Take first n_examples samples
        X = X[:n_examples, :]
        y = y[:n_examples, :]
    else:
        # Take all the positive samples with a number of random negatives
        if n_examples < 0:
            Xt = X[(y == 1).ravel()]
            yt = y[(y == 1).ravel()]
            indices = random.sample(range(X.shape[0]), -1 * n_examples)
            X = np.append(Xt, X[indices], axis=0)
            y = np.append(yt, y[indices], axis=0)
    print("X.shape", X.shape, " y.shape", y.shape)
    print("Fraud: {0:.3f}% {1}".format(len(y[y == 1])*100/X.shape[0], len(y[y == 1])))
    print("Valid: {0:.3f}% {1}".format(len(y[y == 0]) * 100 / X.shape[0], len(y[y == 0])))
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=random_state, stratify=y)
    return Xtrain, Xtest, ytrain, ytest

# data = load_creditcard(-5000) # Take all true samples + 5000 of the others
# data = load_creditcard(5000)  # Take the first 5000 samples
data = load_creditcard(-1000) # Take all the samples

Xtrain = data[0]
Xtest = data[1]
ytrain = data[2]
ytest = data[3]
# Set weights inverse to its count class in dataset
weights = np.ones(Xtrain.shape[0],) * 1.00244
weights[ytrain==1] = 1.99755 

Fraud: 0.173% 492
Valid: 99.827% 284315
X.shape (1492, 28)  y.shape (1492,)
Fraud: 33.177% 495
Valid: 66.823% 997


# Tests

## Test smple_weights
Compute accuracy with weights in samples. The weights are set based on the inverse of the number of samples of each class

In [5]:
C = 23
print("Accuracy of Train without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtrain, ytrain))
print("Accuracy of Train with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtrain, ytrain))
print("Accuracy of Tests without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtest, ytest))
print("Accuracy of Tests with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtest, ytest))

Accuracy of Train without weights 0.9770114942528736
Accuracy of Train with    weights 0.9818007662835249
Accuracy of Tests without weights 0.953125
Accuracy of Tests with    weights 0.9419642857142857


## Test accuracy with different kernels
Compute accuracy on train and test set with default hyperparmeters of every kernel

In [6]:
random_state=1
for kernel in ['linear', 'rbf', 'poly']:
    now = time.time()
    clf = Stree(C=7, kernel=kernel, random_state=random_state).fit(Xtrain, ytrain)
    accuracy_train = clf.score(Xtrain, ytrain)
    accuracy_test = clf.score(Xtest, ytest)
    time_spent = time.time() - now
    print(f"Time: {time_spent:.2f}s\tKernel: {kernel}\tAccuracy_train: {accuracy_train}\tAccuracy_test: {accuracy_test}")


Time: 0.20s	Kernel: linear	Accuracy_train: 0.9712643678160919	Accuracy_test: 0.9575892857142857
Time: 0.09s	Kernel: rbf	Accuracy_train: 0.9932950191570882	Accuracy_test: 0.9620535714285714
Time: 0.09s	Kernel: poly	Accuracy_train: 0.9904214559386973	Accuracy_test: 0.9508928571428571


## Test diferent values of C

In [7]:
t = time.time()
for C in (.001, .01, 1, 5, 17):
    clf = Stree(C=C, random_state=random_state)
    clf.fit(Xtrain, ytrain)
    print(f"************** C={C} ****************************")
    print(f"Classifier's accuracy (train): {clf.score(Xtrain, ytrain):.4f}")
    print(f"Classifier's accuracy (test) : {clf.score(Xtest, ytest):.4f}")
    print(clf)
    print(f"**************************************************")
print(f"{time.time() - t:.4f} secs")

************** C=0.001 ****************************
Classifier's accuracy (train): 0.9550
Classifier's accuracy (test) : 0.9509
root
root - Down
root - Down - Down, <cgaf> - Leaf class=1 belief= 0.980583 counts=(array([0, 1]), array([  6, 303]))
root - Down - Up, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([1]))
root - Up
root - Up - Down
root - Up - Down - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([2]))
root - Up - Down - Up, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([2]))
root - Up - Up, <cgaf> - Leaf class=0 belief= 0.943836 counts=(array([0, 1]), array([689,  41]))

**************************************************
************** C=0.01 ****************************
Classifier's accuracy (train): 0.9569
Classifier's accuracy (test) : 0.9576
root
root - Down
root - Down - Down, <cgaf> - Leaf class=1 belief= 0.990228 counts=(array([0, 1]), array([  3, 304]))
root - Down - Up, <pure> - Leaf class=0 belief= 1.0000

In [8]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
scaler = StandardScaler()
cclf = CalibratedClassifierCV(base_estimator=LinearSVC(), cv=5)
cclf.fit(Xtrain, ytrain)
res = cclf.predict_proba(Xtest)
print(res[:4, :])

[[0.88204928 0.11795072]
 [0.8640131  0.1359869 ]
 [0.94207521 0.05792479]
 [0.90219947 0.09780053]]


## Test iterator
Check different weays of using the iterator

In [9]:
#check iterator
for i in list(clf):
    print(i)

root
root - Down
root - Down - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([314]))
root - Down - Up
root - Down - Up - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([1]))
root - Down - Up - Up, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([12]))
root - Up
root - Up - Down, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([1]))
root - Up - Up
root - Up - Up - Down, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([2]))
root - Up - Up - Up
root - Up - Up - Up - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([1]))
root - Up - Up - Up - Up
root - Up - Up - Up - Up - Down, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([2]))
root - Up - Up - Up - Up - Up
root - Up - Up - Up - Up - Up - Down
root - Up - Up - Up - Up - Up - Down - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([3]))
root - Up - Up - Up - Up - Up - Down - Up, 

In [10]:
#check iterator again
for i in clf:
    print(i)

root
root - Down
root - Down - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([314]))
root - Down - Up
root - Down - Up - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([1]))
root - Down - Up - Up, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([12]))
root - Up
root - Up - Down, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([1]))
root - Up - Up
root - Up - Up - Down, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([2]))
root - Up - Up - Up
root - Up - Up - Up - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([1]))
root - Up - Up - Up - Up
root - Up - Up - Up - Up - Down, <pure> - Leaf class=0 belief= 1.000000 counts=(array([0]), array([2]))
root - Up - Up - Up - Up - Up
root - Up - Up - Up - Up - Up - Down
root - Up - Up - Up - Up - Up - Down - Down, <pure> - Leaf class=1 belief= 1.000000 counts=(array([1]), array([3]))
root - Up - Up - Up - Up - Up - Down - Up, 

## Test STree is a sklearn estimator

In [12]:
# Make checks one by one
c = 0
checks = check_estimator(Stree(), generate_only=True)
for check in checks:
    c += 1
    print(c, check[1])
    check[1](check[0])

1 functools.partial(<function check_no_attributes_set_in_init at 0x12bd3b5f0>, 'Stree')
2 functools.partial(<function check_estimators_dtypes at 0x12bd31710>, 'Stree')
3 functools.partial(<function check_fit_score_takes_y at 0x12bd315f0>, 'Stree')
4 functools.partial(<function check_sample_weights_pandas_series at 0x12bd21ef0>, 'Stree')
5 functools.partial(<function check_sample_weights_not_an_array at 0x12bd2d050>, 'Stree')
6 functools.partial(<function check_sample_weights_list at 0x12bd2d170>, 'Stree')
7 functools.partial(<function check_sample_weights_shape at 0x12bd2d290>, 'Stree')
8 functools.partial(<function check_sample_weights_invariance at 0x12bd2d3b0>, 'Stree')
9 functools.partial(<function check_estimators_fit_returns_self at 0x12bd37710>, 'Stree')
10 functools.partial(<function check_estimators_fit_returns_self at 0x12bd37710>, 'Stree', readonly_memmap=True)
11 functools.partial(<function check_complex_data at 0x12bd2d560>, 'Stree')
12 functools.partial(<function check_dt

In [11]:
# Check if the classifier is a sklearn estimator
from sklearn.utils.estimator_checks import check_estimator
check_estimator(Stree())